In [6]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from random import seed, randrange
from math import sqrt

In [10]:
df = pd.read_csv("Final_Data_pre.csv")
df.head(1)

,league = World: FIFA World Cup,league = Europe: Euro,league = South America: Copa America,team_1,team_2,avgRat_team1,avgRat_team2,avgAge_team1,avgAge_team2,team1_Population,team1_GDP,team1_avg_years_of_schooling,team2_Population,team2_GDP,team2_avg_years_of_schooling,result
0,1.0,0.0,0.0,0.0,0.0,5.097443,1.859638,-3.190945,0.790466,0.291918,0.752425,1.542243,-0.303294,-0.611643,-0.409598,Team1_Win


In [11]:
df.values

array([[1.0, 0.0, 0.0, ..., -0.6116434948705063, -0.4095977152802895,
        'Team1_Win'],
       [1.0, 0.0, 0.0, ..., -0.6935137876059352, -0.4696689218848015,
        'Team2_Win'],
       [1.0, 0.0, 0.0, ..., -0.7340351939106379, -0.6713853031533613,
        'Team1_Win'],
       ...,
       [1.0, 0.0, 0.0, ..., -0.166022912812661, 1.0317699295541083,
        'Draw'],
       [1.0, 0.0, 0.0, ..., 1.2653305879500512, 1.1170983480264265,
        'Team1_Win'],
       [1.0, 0.0, 0.0, ..., 1.0774997552350578, 1.4345200647434495,
        'Draw']], dtype=object)

In [21]:
def cross_validation(dataset, algorithm, n_folds, *args):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for _ in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    folds = dataset_split
    
    scores = list()
    for n in range(len(folds)):
        print(n)
        train_set = list()
        for i in range(len(folds)):
            if i!=n:
                train_set += folds[i]
        test_set = list()
        for row in folds[n]:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        correct = 0
        for i in range(len(actual)):
            if actual[i] == predicted[i]:
                correct += 1
        accuracy = correct / float(len(actual)) * 100.0
        scores.append(accuracy)
    return scores

def euclidean_distance(row1, row2):
    distance = 0.0
    for i in range(len(row1)-1):
        distance += (row1[i] - row2[i])**2
    return sqrt(distance)

def get_neighbors(train, test_row, num_neighbors):
    distances = list()
    for train_row in train:
        dist = euclidean_distance(test_row, train_row)
        distances.append((train_row, dist))
    distances.sort(key=lambda tup: tup[1])
    neighbors = list()
    for i in range(num_neighbors):
        neighbors.append(distances[i][0])
    return neighbors

def k_nearest_neighbors(train, test, num_neighbors):
    predictions = list()
    for row in test:
        neighbors = get_neighbors(train, row, num_neighbors)
        output_values = [trow[-1] for trow in neighbors]
        output = max(set(output_values), key=output_values.count)
        predictions.append(output)
    return predictions

In [25]:
n_folds = 5
num_neighbors = 5
scores = cross_validation(df.values, k_nearest_neighbors, n_folds, num_neighbors)

In [28]:
scores, "Accuracy: " + str(sum(scores)/float(len(scores)))

([35.843054082714744,
  39.660657476139974,
  36.79745493107105,
  35.52492046659597,
  49.94697773064687],
 'Accuracy: 39.554612937433724')